## Discriminated unions

One of the key aspects of F\# (and other functional languages) is that it provides a specific syntax to model those characteristics of inputs and outputs that belong to a well predefined collection. These are called _discriminated unions_. 

The type for our food `vendingMachine` would be:

In [7]:
type FoodProduct =
    | Chips
    | Chocolate
    | Candy 

while for the electronics we could have:

In [8]:
type Electronics = 
    | Phones
    | Speakers
    | Headphones

> 🔔 It is customary to use `PascalCase` for types (i.e., `FoodProduct`).

> ❗️ Case types in a discriminated union should start with an uppercase letter (`Phones`, `Chips`, etc.).

In the expressions above we are defining a type with a name (`FoodProduct`, `Electronics`) that can have several  case types. It is important to stress the fact that the cases of each discriminated union type are _disjoint_, that is, can not be accessed at the same time. For example:

In [9]:
let d = FoodProduct.Chips
let s = Electronics.Speakers

The languages uses the dot `.` to represent a case of a discriminated union. The value `s` represents of course an speaker, and since it is inmutable, there is no possible way that can be a phone or a headphone. 

So how can we define functions with discriminated unions? Let us write a function `price` that receives an input of type `Electronics` and gives us the price:

In [10]:
let price electronic = 
    match electronic with 
    | Phones -> 435
    | Speakers -> 29
    | Headphones -> 122

printfn "The price of speaker is: %A $" (price s)    

The price of speaker is: 29 $


The way one can disaggregate the different cases of an input that is a discriminated union is through _pattern matching_, represented by the construct `match ... with` and then, all the cases. The syntax is pretty straightforward: for each discriminated union case label (after the `|` sign in the construct) the function returns the price as an `int`. 

Note also that the match should contain _all_ the possible cases of the discriminated union. If one is forgetting some case, the compiler will tell us about with some wiggled underlining at the match. This means that the patter matching is _exhaustive_. 

Another very important aspect of the pattern matching is that is evaluated _in the order_ is written. 

But, what if we want to write down a `priceFood` function, and asign a 1.5 price to all the items except Chocolates, that are tagged at a 2.35 price? The language introduces the _wildcard_ symbol that matches _any_ input in the pattern matching construct. The wildcard is represented by the `_` (single underscore) symbol:

In [11]:
let priceFood food = 
    match food with 
    | Chocolate -> 2.35
    | _ -> 1.5 

printfn "Chocolate: %A $" (priceFood FoodProduct.Chocolate)        
printfn "Chips: %A $" (priceFood FoodProduct.Chips)        
printfn "Candy: %A $" (priceFood FoodProduct.Candy)        

Chocolate: 2.35 $
Chips: 1.5 $
Candy: 1.5 $


Here we see the interplay between the order evaluation of the pattern matching and the wildcard. We are returning a specific values for _some cases_ (Chocolate), and asign a common value for _the rest of the cases_. When a food `value` is received by `priceFood` it is compared with the `Chocolate` case first. If it is not chocolate, then is get captured by the wildcard pattern. Consider the following example:

In [12]:
let priceSale food = 
    match food with 
    | _ -> 1.5 
    | Chocolate -> 2.35

printfn "Chocolate: %A $" (priceSale FoodProduct.Chocolate)        
printfn "Chips: %A $" (priceSale FoodProduct.Chips)        
printfn "Candy: %A $" (priceSale FoodProduct.Candy)        

Chocolate: 1.5 $
Chips: 1.5 $
Candy: 1.5 $


Well, everything is "on sale" at 1.5! That is because the wildcard captures _any input_ and since it is the first _case_, any food will be at that price. 

Here again, the compiler behind scenes comes to our rescue. It will let you know that some of the pattern matching rules will not be reached:

![This will save you from bankruptcy](../figures/rule_will_never_match.png)


### Combining discriminated unions and basic types

In many cases, the discriminated union construct is not general enough, so one can combine it with basic types. Let us say that we want to identify the brand of each of our items in the vending machines. Since it can accomodate multiple brands of products, we decide that we represent the brand by a `string`. We can expand our `FoodProduct` discriminated union type as:


In [13]:
type BrandedFood =
    | Chips of string 
    | Chocolate of string 
    | Candy of string 

Each of the cases of the `BrandedFood` type has now a _value_ of type `string`. The discriminated union makes use of the keyword `of` to associate each case with each value type. One can define identifiers for this compound type as:

In [14]:
let belovedChocolate = BrandedFood.Chocolate "Wonka"
let healthyChips = BrandedFood.Chips "NotALays"
let sourCandy = BrandedFood.Candy "TearDrops"

In the brand example, we choose to combine all the cases with the same basic type, `string`. But, again, one can mix and match. For example, a chocolate can come in different presentations:

In [15]:
type ChocolatePresentation =
| Bar of float  // a chocolate Bar of a given weight
| Box of int // a package with a number of chocolate pieces    

Or, if we want to model the change money the vending machine returns to the customer, we can define a case where an actual amount is returned, and another one that represents the fact that the client just put the exact money into the machine:

In [16]:
type Change =
| Amount of float
| NoChange

Pattern matching in functions against these compound discriminated unions is again fairly simple. To get the brand of a `BrandedFood` product, we can define the function `brand`:

In [17]:
let brand product = 
    match product with
    | Chips p -> p 
    | Chocolate p -> p 
    | Candy p -> p  

In [18]:
printfn "Brand of belovedChocolate: %s" (brand belovedChocolate)

Brand of belovedChocolate: Wonka


In this function, each case has an associated value represented by the identifier `p`, that is _unwrapped_ from the discriminated union, and returned. 

Another example:

In [19]:
let changeValue change =
    match change with 
    | Amount money -> money
    | NoChange -> 0 

let c = Amount 3 

printfn "You are receiving %A $ as change" (changeValue c)

You are receiving 3.0 $ as change


> Note on how a discriminated union that mix types presents a kind of divergence in code. This is refered to inputs and outputs: if a function receives one of these du as an input, it needs to wrap the du type into another type, because it is not possible for a function to return the wrapped value of a du, since it can be different types. 

### Protecting inputs with single discriminated unions










- Say something about validation?







#### Comment on FP vs OOP ?


 
On the contrary, methods in a class are designed to change (that is, _mutate_) values of an instance of that class. 
[FP vs OP](https://betterprogramming.pub/functional-programming-vs-object-oriented-programming-why-you-dont-need-to-choose-29820f89f348)

We also presented the primitive types as an example, but to program with a reasonable chance of success, it is necessary for the language to be able to aggregate different primitive types in more complex ones. In the case of object oriented languages, one builds a hierarchy of classes that represent the domain one is working on. Those classes encapsulate not only data (that is described in types) but also their associated behavior through methods. In the case of functional F\#, and other functional languages, there is no need to attach behavior to types.





#### Thinking with types

As said before, almost all programming languages provide an idea of aggregating simple types (strings, bools, ints, etc.) into more complex ones, which is needed to properly describe a domain. The basic data of a user (name, email, etc.) can be put together in a type, the email data (to, from, subject, text, etc.) in another one and so on. Combining these new types one can generate more and more complex ones, until you can build, for example, all the types needed for an email application. 

One of the most important steps in programming is to study the domain one is working on, and to fully understand how one can describe that domain with data and the processes that manage those data. In object oriented programming one is allowed (even required) to glue related processes and data together in classes. In functional programming one focuses on how processes (that is, functions) relate different kinds of data (that is, types). Since data in functional programming is inmutable and functions create outputs from inputs, one needs to identify the processes of our domain and distill them into functions, with proper inputs and outputs, in such a way that one can combine those functions together by composing them properly. Therefore, the analysis of a process and it conversion into functions not only requires to define the rules and the logic of the creation of outputs from inputs, buy more importantly, to adequate stablish those inputs and outputs such that composition of functions can be used (see again this [beautiful picture example](https://mathinsight.org/function_machine_composition)).

Functional programming languages typically define two kinds of aggregate types: product types and sum types. Product types 

#### Product type: Records





#### Sum type: Discriminated unions